This is the code implementation of **NegGCNs**, which can be run directly on Google colab.

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

import torch
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 2.6MB 2.6MB/s 
     |████████████████████████████████| 1.4MB 2.5MB/s 
     |████████████████████████████████| 225kB 5.2MB/s 
     |████████████████████████████████| 235kB 19.4MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 


# 0. Functions

In [2]:
from collections import defaultdict
import numpy as np
def load_item_pop(X_train):
    item_pop = list()
    node_deg = dict()
    dd = defaultdict(list)
    #edge: <node1> <node2>
    for edge in X_train:
        #   <node2>       <node1> 
        dd[int(edge[1])].append(int(edge[0]))
    for key in dd.keys():
        item_pop.append(1)
    #deg_sum : nodes number
    deg_sum = np.sum(item_pop)
    for key in dd.keys():
        node_deg[key] = 1/deg_sum
    return node_deg, dd

In [3]:
import time
from collections import defaultdict

#DFS
class Personalized():
    def __init__(self, nx_G, mask, walks_num):
        self.G = nx_G
        self.mask = mask
        self.walks_num = walks_num

    # iterative version
    # walks_num = 10
    def dfs(self, start_node):
        stack=[]
        stack.append(start_node)
        seen=set()
        seen.add(start_node)
        walks = []
        mask_list = set(self.mask[start_node])
        # print("mask_list", mask_list)
        while (len(stack)>0):
            vertex=stack.pop()
            # print("vertex", vertex)
            nodes=self.G[vertex]
            # print("nodes", nodes)
            for w in nodes:
                if w not in seen:
                    stack.append(w)
                    seen.add(w)
            # print("stack",stack)
            # if vertex in mask_list:
            #     pass
            # else:
            walks.append(vertex)
            # print("walks",walks)
            if len(walks) >= self.walks_num:
                break
        return walks

    # #recursiveche version
    # def dfs(self, start_node, walks=[]):
    #     walks.append(start_node)
    #     mask_list = set(self.mask[start_node])
    #     for w in self.G[start_node]:
    #         if w not in walks:
    #             if len(walks) >= self.walks_num:
    #                 break
                
    #             if w in mask_list:
    #                 pass
    #             else:
    #                 if w == start_node:
    #                     pass
    #                 else:
    #                     dfs(G, w, walks)
    #     return walks
    
    def intermediate(self):
        i = 0 
        candidate = defaultdict(list)
        for node in self.G.nodes():
            # print(node)
            walk = self.dfs(node)
            candidate[node].extend(walk)
            # i=i+1
            # if i==2:
            #   break
        return candidate


def candidate_choose(nx_Graph, mask, walks_num):
    G = Personalized(nx_Graph, mask, walks_num)
    #defaultdict(<class 'list'>, {967: [1186, 1222, 1145, 1119, 1453, 1112, 1131, 1084, 1834, 1295]})
    candidates = G.intermediate()
    return candidates

In [4]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [5]:
import torch
import torch.nn.functional as F

class NegaGCN(torch.nn.Module):
    def __init__(self, hidden_channels,candidates,start_given,q_1_dict,N_steps,node1,NegRate,index_selectindex):
        super(NegaGCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.negaconv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
        self.negaconv2 = GCNConv(hidden_channels, dataset.num_classes)
        self.candidates = candidates
        self.start_given = start_given
        self.q_1_dict = q_1_dict
        self.N_steps = N_steps
        self.node1 = node1
        self.NegRate = NegRate
        self.index_selectindex = index_selectindex

    def forward(self, x, edge_index):
        #---first layer----
        posi_x1 = self.conv1(x, edge_index)
        #nega_sample: perform negative sampling
        #data type: list
        nega_sample = self.negative_sampling(posi_x1,x, edge_index.shape[1])
        
        #get negative sampling node index
        nega_sample = torch.LongTensor(nega_sample)
        NegOut = edge_index[0,index_selectindex].unsqueeze(0)
        NegIn = nega_sample[index_selectindex].unsqueeze(0)
        NegaIndex = torch.cat((NegOut, NegIn), 0)

        #Using negative index to perfom conv
        nega_x1 = self.negaconv1(x, NegaIndex)
        #positve - negative
        final_x1 = posi_x1 - self.NegRate * nega_x1
        final_x1 = final_x1.relu()
        final_x1 = F.dropout(final_x1, p=0.5, training=self.training)

        #---second layer----

        posi_x2 = self.conv2(final_x1, edge_index)
        nega_sample = self.negative_sampling(posi_x2,final_x1, edge_index.shape[1])

        nega_sample = torch.LongTensor(nega_sample)
        NegOut = edge_index[0,index_selectindex].unsqueeze(0)
        NegIn = nega_sample[index_selectindex].unsqueeze(0)
        NegaIndex = torch.cat((NegOut, NegIn), 0)

        nega_x2 = self.negaconv2(final_x1, NegaIndex)
        final_x2 = posi_x2 - self.NegRate * nega_x2
        return final_x2
    
    def negative_sampling(self,posi,x,num_edges):
        # distribution = [i/np.sum(distribution) for i in distribution]
        if self.start_given is None:
            start = np.random.choice(list(self.q_1_dict.keys()),num_edges)  # random init nodes 
        else:
            start = self.start_given
        #print(start)
        count = 0
        cur_state = start #index of input node
        user_list = self.node1
        walks = defaultdict(list)
        generate_examples = list()
        while True:
            y_list = list()
            q_probs_list = list()
            #(1) Generate y ∼ q(y|X(t)) where the proposal distribution q is arbitrary chosen as long as positive everywhere.
            q_probs_next_list = list()
            count += 1
            sample_num = np.random.random() 
            #print(sample_num)
            if sample_num < 0.5: # sample proba < 0.5 negtive correlation q_probs = 0.01
                #q_1_dict : 1/deg_sum                        len(cur_state)=batch size=512   Same probability for each point
                y_list = np.random.choice(list(self.q_1_dict.keys()), len(cur_state), p=list(self.q_1_dict.values()))
                #print("y_list",y_list)
                #q(i,j) Same probability for each point
                q_probs_list = [self.q_1_dict[i] for i in y_list]
                #print("q_probs_list",q_probs_list)
                #q(j,i) Same probability for each point
                q_probs_next_list = [self.q_1_dict[i] for i in cur_state]
                #print("q_probs_next_list",q_probs_next_list)
            else:
                for i in cur_state:
                    distribution = [1/len(self.candidates[i])] * len(self.candidates[i])
                    #print('i',i)
                    y = np.random.choice(self.candidates[i], 1, p=distribution)[0]
                    #print("y",y)
                    y_list.append(y)
                    index = self.candidates[i].index(y) # index of y in candidates[i]
                    #print("index",index)
                    q_probs = distribution[index]
                    #print("q_probs",q_probs)
                    q_probs_list.append(q_probs)
                    node_list_next = self.candidates[y]
                    #print("node_list_next",node_list_next)
                    if i in node_list_next:
                        index_next = node_list_next.index(i)
                        q_probs_next = distribution[index_next]
                    else:
                        q_probs_next = self.q_1_dict[i]
                    q_probs_next_list.append(q_probs_next) 
                    #print("q_probs_next_list",q_probs_next_list)

            q_probs_list = torch.Tensor(q_probs_list)
            q_probs_next_list = torch.Tensor(q_probs_next_list)
            #print(y_list)
            user_list_value = posi[user_list]
            #print(user_list_value)
            cur_state_value = posi[cur_state]
            #print(cur_state_value)
            y_list_value = posi[y_list]
            #print(y_list_value)
            p_probs = torch.sigmoid(torch.pow(torch.sum(user_list_value * y_list_value, axis=1), 0.25))
            p_probs_next = torch.sigmoid(torch.pow(torch.sum(user_list_value * cur_state_value, axis=1), 0.25))
            u = np.random.rand()
            # print("u",u)
            p_probs = p_probs
            p_probs_next = p_probs_next
            A_a_list = (p_probs * q_probs_next_list)/(p_probs_next * q_probs_list)
            A_a_list = A_a_list.detach().numpy()
            # print(A_a_list)
            next_state = list()
            next_user = list()
            # N_steps = 10 
            if count > self.N_steps:
                for i in list(range(len(cur_state))):
                  walks[user_list[i]].append(y_list[i])
   
            else:# count < 10
                for i in list(range(len(cur_state))):
                  A_a = A_a_list[i]
                  #   alpha                    
                  alpha = min(1, A_a)
                  if u < alpha:
                    #accept
                    next_state.append(y_list[i])
                  else:
                    next_state.append(cur_state[i])
                cur_state = next_state
                #print("next_state", next_state)
            length = 0
            for key in walks.keys():
                length += len(walks[key])
            #print(length)

            if length == num_edges:
                # print("count", count)
                generate_examples = list()
                for user in node1:
                    d = walks[user]
                    if len(d) == 1:
                        generate_examples.append(d[0])    
                    else:
                        generate_examples.append(d[0])
                        del walks[user][0]
                break
            else:
                continue  
        return generate_examples

# def __init__(self, hidden_channels,candidates,start_given,q_1_dict,N_steps,node1):

# 1. Load datasets

In [6]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import KarateClub

dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
# dataset = Planetoid(root='data/Planetoid', name='Pubmed')
#dataset = KarateClub()
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: Citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


# 2. Generate graphs to calculate q() and DFS

In [7]:
import random
from torch_geometric.utils import add_self_loops,to_networkx
G = to_networkx(data, to_undirected=True)

edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.x.size(0))
# print(edge_index.shape)
edge_index_list = torch.transpose(edge_index, 0, 1).tolist()
# print(edge_index_list)
node1 = [x[0] for x in edge_index_list] # out nodes
node2 = [x[1] for x in edge_index_list] # out nodes
# print(len(node1))
# print(len(edge_index_list))
# print(edge_index_list[0])
q_1_dict, mask = load_item_pop(edge_index_list)

walks_num = 10
candidates = candidate_choose(G, mask, walks_num)
# print(candidates[0])

# 3. Get sampling nodes number

In [8]:
rate = 0.9 #sampling rate
select_index = []
outnode = edge_index[0].numpy()
for i in range(data.num_nodes):
  temp = np.where(edge_index[0].numpy()==i)
  index_EdgeIndex = np.squeeze(np.asarray(temp))
  if np.ndim(index_EdgeIndex) == 0:
    select_index.append(list(np.array(index_EdgeIndex, ndmin=1)))
  else:
    EdgeNumber = index_EdgeIndex.shape[0]
    sampleNumber = int(EdgeNumber * rate)
    # list1 = range(EdgeNumber)
    # print(list1)
    randIndex = random.sample(list(index_EdgeIndex), sampleNumber)
    # print(randIndex)
    select_index.append(randIndex)
    # print("select_index",select_index)
# print(select_index)
index_selectindex = sum(select_index, [])
# print(index_selectindex)
# print(len(index_selectindex))

# 4. Training

In [ ]:
def test():
      model.eval()
      out = model(data.x, edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


model = NegaGCN(hidden_channels=16,
          candidates=candidates,
          start_given=None,
          q_1_dict=q_1_dict,
          N_steps=10,
          node1=node1,
          NegRate = 1.25,
          index_selectindex = index_selectindex)

# model = GCN(hidden_channels=16)

print(model)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# data = data.to(device)
# edge_index = edge_index.to(device)

if torch.cuda.is_available():
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()
    
    test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f}')